In [2]:
library(tidyverse)
library(lubridate)
library(forecast)
library(car)
library(boot)
library(zoo)
library(factoextra)
cities <- c(
  "New York", "Los Angeles", "Chicago", "Houston", "Phoenix",
  "Philadelphia", "San Antonio", "San Diego", "Dallas", "San Jose"
)
set.seed(5100)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 4.0.0     ✔ purrr   1.1.0
✔ tibble  3.3.0     ✔ dplyr   1.1.4
✔ tidyr   1.3.1     ✔ stringr 1.5.2
✔ readr   2.1.5     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::%||%()   masks base::%||%()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some



Attaching package: ‘boot’


The following object is masked from ‘package:car’:

    logit



Attaching package: ‘zoo’


The following objects are masked fr

## PM25
### East Cities vs. West Cities

In [ ]:
air_df <- air_df %>%
  mutate(
    Region = case_when(
      City %in% c("New York", "Philadelphia") ~ "East",
      City %in% c("Los Angeles", "San Diego") ~ "West",
      TRUE ~ "Other"
    )
  )

east_pm25 <- air_df %>% filter(Region == "East") %>% pull(PM25)
west_pm25 <- air_df %>% filter(Region == "West") %>% pull(PM25)

ERROR: Error: object 'air_df' not found



#### T-Test

In [ ]:
t.test(east_pm25, west_pm25, var.equal = FALSE)

#### Shapiro Test
```{r}
shapiro.test(sample(east_pm25, 500))  
shapiro.test(sample(west_pm25, 500))
```

Both p-values are way below 0.05, so we reject the null hypothesis of normality. both distributions are not normally distributed. So we can't trust a standard t-test, so instead we will use Wilcoxon rank-sum test.

#### Wilcox Test

```{r}
wilcox.test(east_pm25, west_pm25)
```

Since the p-value is way below 0.05, you reject the null hypothesis and this tells us there is a statistically significant evidence that PM2.5 levels between East and West cities are different. We can see the median values to see which region has higher PM2.5:

#### Median PM2.5 Levels

```{r}
median(east_pm25, na.rm = TRUE)
median(west_pm25, na.rm = TRUE)
```

The difference is not huge it is slight, but with the wildfires, we would expect this same pattern and even more exacerbated that the west has higher pm25 levels.

### Individual Cites

#### Levene's Test

```{r}
# Check equal variances first
leveneTest(PM25 ~ City, data = air_df)

anova_fit <- aov(PM25 ~ City, data = air_df)
summary(anova_fit)

```

The null hypothesis tells us that Levene's test is testing that all groups have equal variances. Since the p value is way below 0.05, we reject the null hypothesis which means variances differ significantly across the 10 cities. Homogeniety of variances is violated.

#### Kruskal Test

```{r}
kruskal.test(PM25 ~ City, data = air_df_clean)
```

Because both normality and equal variances assumptions were violated, we used Kruskal-Wallis test to compare PM2.5 levels across the ten major cities. The test is showing a highly significant result as the p-value is way below 0.05. This shows then that the PM2.5 concentration levels differ substantially across these cities. There is a meaningful variation in urban air quality patterns across the country.

### LA vs. NY
#### Bootstrapping
```{r}
la_ny_pm25 <- air_df_clean %>%
  filter(City %in% c("Los Angeles", "New York"))
boot_stat_pm25 <- function(data, indices) {
  d <- data[indices, ]
  mean_la  <- mean(d$PM25[d$City == "Los Angeles"], na.rm = TRUE)
  mean_ny  <- mean(d$PM25[d$City == "New York"],   na.rm = TRUE)
  mean_la - mean_ny
}

boot_res_pm25 <- boot(data = la_ny_pm25,
                      statistic = boot_stat_pm25,
                      R = 2000)

boot_res_pm25
boot.ci(boot_res_pm25, type = c("perc", "bca"))
```

## Ozone (O3)
now testing for ozone across all 10 cities as we did for PM2.5.

### Individual Cities

#### Levene Test

```{r}
leveneTest(O3 ~ City, data = air_df_clean)
```

Same as PM2.5, p-value is less than 0.05 so ANOVA assumption is violated, we must use kruskal-wallis.

#### ANOVA
```{r}
anova_o3 <- aov(O3 ~ City, data = air_df_clean)
summary(anova_o3)
```

The p-value here is small so although evidence may show O3 levels vary across cities, we can't trust ANOVA since the assumptions were violated.

#### Kruskal Test
```{r}
kruskal.test(O3 ~ City, data = air_df_clean)
```

P-value is also way below 0.05, so median O3 levels differ significantly across cities.

#### Median O3 Levels 
```{r}
air_df_clean %>%
  group_by(City) %>%
  summarise(median_O3 = median(O3, na.rm = TRUE)) %>%
  arrange(desc(median_O3))
```


### LA vs. NY
#### Wilcox Test

```{r}
# la_o3 <- air_df_clean %>% filter(City == "Los Angeles") %>% pull(O3)
# ny_o3 <- air_df_clean %>% filter(City == "New York") %>% pull(O3)

# wilcox.test(la_o3, ny_o3)
```

```{r}
wilcox.test(
  air_df_clean$O3[air_df_clean$City == "Los Angeles"],
  air_df_clean$O3[air_df_clean$City == "New York"]
)
```
#### Bootstrapping
```{r}
la_ny_o3 <- air_df_clean %>%
  filter(City %in% c("Los Angeles", "New York"))
boot_stat_o3 <- function(data, indices) {
  d <- data[indices, ]
  mean_la <- mean(d$O3[d$City == "Los Angeles"], na.rm = TRUE)
  mean_ny <- mean(d$O3[d$City == "New York"], na.rm = TRUE)
  mean_la - mean_ny
}
boot_res_o3 <- boot(data = la_ny_o3,
                    statistic = boot_stat_o3,
                    R = 2000)

boot_res_o3
boot.ci(boot_res_o3, type = c("perc", "bca"))

```


## Nitrogen Dioxide (NO2)
### Individual Cities   
#### Levene's Test
```{r}
leveneTest(NO2 ~ City, data = air_df_clean)
```

#### ANOVA
```{r}
anova_no2 <- aov(NO2 ~ City, data = air_df_clean)
summary(anova_no2)
```

#### Kruskal Test
```{r}
kruskal.test(NO2 ~ City, data = air_df_clean)
```

### Chicago vs. Houston
#### Wilcox Test
```{r}
chi_hou_no2 <- air_df_clean %>% 
  filter(City %in% c("Chicago", "Houston"))

wilcox.test(NO2 ~ City, data = chi_hou_no2)

```
#### Bootstrapping
```{r}
boot_stat_no2 <- function(data, indices) {
  d <- data[indices, ]
  mean_chicago <- mean(d$NO2[d$City == "Chicago"], na.rm = TRUE)
  mean_houston <- mean(d$NO2[d$City == "Houston"], na.rm = TRUE)
  mean_chicago - mean_houston
}

boot_res_no2 <- boot(data = chi_hou_no2,
                     statistic = boot_stat_no2,
                     R = 2000)

boot_res_no2
boot.ci(boot_res_no2, type = c("perc", "bca"))

```

> \
> To quantify differences in nitrogen dioxide (NO₂) exposure between Chicago and Houston, we conducted a nonparametric bootstrap with 2,000 resamples. The observed difference in mean NO₂ was 2.58 ppb, with Chicago exhibiting higher levels.
>
> The 95% percentile bootstrap interval (1.36, 3.87) and the BCa interval (1.45, 3.95) both excluded zero, indicating strong statistical evidence that Chicago’s mean NO₂ concentration is higher than Houston’s.
>
> This difference is not only statistically significant but also meaningful in a public-health context. NO₂ differences of 2–4 ppb have been associated with increased risks of asthma exacerbations, respiratory stress, and emergency department visits. The findings reflect underlying differences in traffic density, atmospheric conditions, and industrial structure between the two metropolitan areas.

#### Plot
```{r}
air_df_clean %>%
  filter(City %in% c("Chicago", "Houston")) %>%
  mutate(Month = month(Date, label = TRUE)) %>%
  group_by(City, Month) %>%
  summarise(mean_NO2 = mean(NO2, na.rm = TRUE)) %>%
  ggplot(aes(Month, mean_NO2, group = City, color = City)) +
  geom_line(size = 1.2)

```

Warm air → more vertical mixing → pollution disperses\

Cold air → less mixing → pollution stays trapped near surface

#### Weekends vs. Weekdays

```{r}
air_df_clean <- air_df_clean %>%
  mutate(Weekday = wday(Date, label = TRUE, week_start = 1))
air_df_clean %>%
  filter(City %in% c("Chicago", "Houston")) %>%
  group_by(City, Weekday) %>%
  summarise(mean_NO2 = mean(NO2, na.rm = TRUE)) %>%
  ggplot(aes(Weekday, mean_NO2, color = City, group = City)) +
  geom_line(size = 1.2) +
  labs(title = "Weekday vs Weekend NO₂ Levels", y = "Mean NO₂ (ppb)") +
  theme_minimal(base_size = 14)

```

```{r}
air_df_clean <- air_df_clean %>%
  mutate(
    day_type = ifelse(Weekday %in% c("Sat", "Sun"), "Weekend", "Weekday")
  )
chi_week <- air_df_clean %>% filter(City == "Chicago")
wilcox.test(NO2 ~ day_type, data = chi_week)
hou_week <- air_df_clean %>% filter(City == "Houston")
wilcox.test(NO2 ~ day_type, data = hou_week)
```

for Chicago, it is not statistically significant at 5 percent. For Houston, it is highly significant. houston shows strong weekday \> weekend NO2. So, NO2 rises with weekday commuting and falls on weekends. Traffic drives NO₂, but *meteorology (weather, winds, etc) determines how much* it shows up in air quality.